In [1]:
# pylint: disable-all
main_path = 'e://ehz'
import sys
import os
sys.path.append(main_path)
sys.path.append('e://stripping/KR_example-main/src')
os.chdir(main_path)


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(rc = { 'figure.figsize': (15, 5) })


import utils
#import fut_pricer

import kr_model
import kr_utils

import plotly.express as px

In [2]:
P = utils.get_bond_prices(bond_type='us', ptype='mid')
bonds = utils.get_bonds()
bonds['length'] = bonds['length'].apply(lambda x: x if x!=31 else 30)
pyd = utils.get_bonds_pyd()
#D = utils.get_bond_duration(ptype='calc')
Y = utils.get_bond_yields(bond_type='us', ptype='mid')

In [3]:
def get_bond_position(x, basket):
    return basket[basket['length'] == x.length].sort_values(by = 'issue_dt', ascending = False).index.get_loc(x.name)

def roundPartial (value, resolution):
    return round (value / resolution) * resolution

def custom_bond_position(x, basket, R):
    return basket[
                basket['ttm'].apply(lambda ttm: roundPartial(ttm, R)) == 
                roundPartial(x.ttm, R)
                ].sort_values(by = 'issue_dt', ascending = False).index.get_loc(x.name)


def construct_C (ttm,  pyd,  bonds,  date):     
    # Matrix defining al1 future flows:     
    # [i,j] = flow at time tj for bond i
    max_ttm = ttm.max()    
    n_bonds = len(ttm)
    C = np.zeros((n_bonds, int(max_ttm)))
    for i  in range(n_bonds):        
        cusip = ttm.index[i]
        coupon = bonds.loc[cusip, 'coupon']
        xij = pyd.loc[[cusip]].apply(lambda x:(x-date).days).values         
        xij = xij[xij>0]
        C[i, xij-1] = coupon/2
        T = xij[-1]
        C[i, T-1] +=100     
    return C

def prep_data(P, C ,density, bonds_density):
    P_array = P.values
    n_bonds = len(P)
    ytm, dur = np.zeros (n_bonds), np.zeros(n_bonds) # YTM and duration
    ttm = np.zeros(n_bonds) # time to maturity in days
    for i in range(n_bonds):
        time_to_cashflow_inday = np.where(C[i]!=0)[0] + 1
        ytm [i] , dur[i] = kr_utils.get_ytm_and_duration(C[i][time_to_cashflow_inday-1], time_to_cashflow_inday, P_array[i])
        ttm [ i ] = max(time_to_cashflow_inday)

    n_tenors = len(density)
    h = (bonds_density*n_tenors).values
    inv_w = (dur*P_array) **2*h
    data = { 'P':P_array, 'C':C, 'ytm':ytm, 'dur':dur, 'ttm':ttm, 'inv_w':inv_w }
    return data
    



act = 365
alpha = 0.05
delta = 0
ridge = 1
ffund = utils.get_repo('ffund')

pyd = utils.get_bonds_pyd()
from tqdm import tqdm



In [4]:
G =pd.read_csv('data/US_TREAS_CURVE/OTR/discount_factors.csv',index_col=0, parse_dates=True)
RATES= pd.read_csv('data/US_TREAS_CURVE/OTR/rates.csv', index_col=0, parse_dates=True)
E=pd.read_csv('data/US_TREAS_CURVE/OTR/errors.csv', index_col=0, parse_dates=True)
#
G.columns=[int(c) for c in G.columns]
RATES.columns=[int(c) for c in RATES.columns]


#G = pd.DataFrame(columns = np.arange(1,31*act+1), dtype='float')
#RATES = pd.DataFrame(columns = np.arange(1,31*act+1), dtype='float')
#E=pd.DataFrame(columns = P.columns, dtype='float')

In [9]:
failed=[]
dates=list(reversed(P.loc[G.index[-1]:].index))

In [ ]:
i=1
for date in tqdm(dates[1:]):

    try : 

        if date in G.index and not all(G.loc[date].isna()):
            continue

        else:
            s_date=P.loc[date:].index[1]
            P0=P.loc[date]
            

            used_date= s_date

            filter1 = bonds['issue_dt']<=date
            filter2 = bonds['maturity']>date
            filter3 = bonds.index.isin(P.columns)

            basket= bonds[filter1 & filter2 & filter3].copy()

            t1=pd.offsets.BusinessDay(1)
            basket['ttm']= (basket['maturity']-t1+t1 - used_date).apply(lambda x:x.days/act)
            basket['age']= (used_date - basket['issue_dt']).apply(lambda x:x.days/act)
            basket['ttm_p'] =100*basket['ttm']/basket['length']
            basket['length_text']=basket['length'].astype(int).astype(str)
            basket['otr_position']=basket.apply(lambda x: get_bond_position(x, basket), axis=1)
            R=0.5
            basket['custom_position'] = basket.apply(lambda x: custom_bond_position(x, basket,R), axis=1)

            filter1 = basket['ttm']>1
            filter2 = basket['ttm_p']>50
            filter3 = basket['otr_position']<1
            filter4 = basket['custom_position']==0

            filters=pd.concat([
                #filter1,
                #filter2,
                filter3,
                #filter4        
                ], axis=1)

            intersec_filters=filters.prod(axis=1).astype (bool)


            basket=basket[intersec_filters].copy()

            
            basket['p_mid']=P0[basket.index]
            basket['AI']=basket.apply(lambda x: utils.calculate_accrued_interest (x.coupon, x.maturity, s_date), axis=1)
            basket['dirty']=basket['AI']+basket['p_mid']


            r1d = ffund.loc[date]
            basket.loc['1D']=pd.Series({
                                        'id_isin': '1D',
                                        'maturity': s_date+pd.offsets.Day(),
                                        'int_acc_dt': None,
                                        'issue_dt': s_date,
                                        'coupon': 0,
                                        'length': 1/act,
                                        'ttm': 1/act,
                                        'length _text': 'ID',
                                        'y_mid': r1d,
                                        'p_mid': 100*np.exp(-r1d*1/act/100),
                                        'AI': 0,
                                        'dirty': 100*np.exp(-r1d*1/act/100)
                                    })


            density=basket.groupby(lambda x: round(basket. loc[x, 'ttm'])) .apply (lambda x: len(x))
            basket_density=basket.apply(lambda x: density.loc[round(x.ttm)], axis=1)

            # P array
            P0=basket['dirty']
            P_array= P0.values
            n_bonds= len(P)
            # term to maturity
            ttm=(basket['ttm']*act)
            ttm_array=ttm.values

            # payment dates
            pyd.loc['1D'] = basket.loc['1D', 'maturity']
            # see docs of 'construct_C'
            C=construct_C(ttm, pyd, basket, s_date)

            data=prep_data(P0, C, density, basket_density)

            # max time to maturity in days
            N=int(ttm.max())
            # generate kerne] matrix
            K=kr_model.generate_kernel_matrix(alpha, delta, N, N)

            #** fit KR model
            # KR ridge penalty term

            dict_fit=kr_model.KR(    C=data['C'], # cashflow matrix
                                    B=data['P'], # price vector
                                    ridge=ridge, # ridge hyper-parameter
                                    inv_w=data['inv_w'], #p.ones like(data['inv w']I. # inverse of the weichting vector
                                    K=K, # kernel matrix.
                                    start_curve=None
            )

                                
            r=dict_fit['y_solved']
            g=dict_fit['g_solved']
            c=data['C']
            M=C.shape[0]
            B_fitted=C@g[:C.shape[1]]
            e=- (data['P']-B_fitted)*100
            e=e/(data['dur']*data['P']/100)
            np.abs(e).mean(), np.abs(e).std()
            e=pd.Series(index=basket.index, data=e)


            G.loc[date, np.arange(1 , len(g)+1)] = g
            RATES.loc[date, np.arange(1 ,len(g)+1)] = r
            for cusip in e.index:
                E.loc[date, cusip]= e.loc[cusip]

            if i%30==0:
                G=G.sort_index()
                G.to_csv('data/US_TREAS_CURVE/OTR/discount_factors.csv')
                RATES=RATES.sort_index()
                RATES.to_csv(f'data/US_TREAS_CURVE/OTR/rates.csv')
                E=E.sort_index()
                E.to_csv('data/US_TREAS_CURVE/OTR/errors.csv')

            i=i+1

    except : 
        print(date)
        playsound(sound_path) 
G=G.sort_index()
G.to_csv('data/US_TREAS_CURVE/OTR/discount_factors.csv')
RATES=RATES.sort_index()
RATES.to_csv(f'data/US_TREAS_CURVE/OTR/rates.csv')
E=E.sort_index()
E.to_csv('data/US_TREAS_CURVE/OTR/errors.csv')


  0%|          | 0/79 [00:00<?, ?it/s]C:\Users\hacen\AppData\Local\Temp\ipykernel_25636\3645580763.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  basket.loc['1D']=pd.Series({
  1%|▏         | 1/79 [00:05<07:31,  5.78s/it]C:\Users\hacen\AppData\Local\Temp\ipykernel_25636\3645580763.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  basket.loc['1D']=pd.Series({
  3%|▎         | 2/79 [00:11<07:06,  5.53s/it]C:\Users\hacen\AppData\Local\Temp\ipykernel_25636\3645580763.py:55: FutureWarning: T

In [22]:
import plotly.express as px
sub_folder='OTR'
E=pd.read_csv(f'data/US_TREAS_CURVE/{sub_folder}/errors.csv', index_col=0, parse_dates=True)
AGG_E= E.T.groupby(lambda x: bonds.loc[x, 'length']if x !='1D' else 0).sum().T
AGG_E['MAE']=AGG_E.abs().mean(axis=1)
AGG_E['MAX']=AGG_E.abs().max(axis=1)
px.line(AGG_E[['MAE', 'MAX']])

## friday

In [15]:
date = pd.to_datetime('03/14/2025')
s_date = pd.to_datetime('03/17/2025')
used_date= s_date

In [19]:
cusips = ['91282CMM', '91282CMP', '912810UG', '91282CMN', '912810UJ', '91282CMR', '91282CMS']
coupon = [4.625, 4.125, 4.625, 4.250, 4.750, 4.125, 3.875]
issue_dt = ['02/18/2025', '02/28/2025', '02/18/2025', '02/18/2025', '02/28/2025', '02/28/2025', '02/17/2025']
maturity = ['02/18/2035', '02/28/2027', '02/18/2055', '02/18/2028', '02/28/2045', '02/28/2032', '02/17/2028']
length = [10, 2, 30, 3, 10, 7, 3]
prices = [102.484375, 100.19726563, 100.03125, 100.65625, 101.2421875, 99.515625, 99.62890625]


P0 = pd.Series(index = cusips, data = prices, dtype=float)
basket = pd.DataFrame(index = cusips, columns = bonds.columns)

basket['coupon'] = coupon
basket['issue_dt'] = [pd.to_datetime(t) for t in issue_dt]
basket['maturity'] = [pd.to_datetime(t) for t in maturity]
basket['length'] = length

basket['p_mid']=P0[basket.index]
basket['AI']=basket.apply(lambda x: utils.calculate_accrued_interest (x.coupon, x.maturity, s_date), axis=1)
basket['dirty']=basket['AI']+basket['p_mid']

t1=pd.offsets.BusinessDay(1)
basket['ttm']= (basket['maturity']-t1+t1 - used_date).apply(lambda x:x.days/act)
#basket['age']= (used_date - basket['issue_dt']).apply(lambda x:x.days/act)
#basket['ttm_p'] =100*basket['ttm']/basket['length']
#basket['length_text']=basket['length'].astype(int).astype(str)
#basket['otr_position']=basket.apply(lambda x: get_bond_position(x, basket), axis=1)
#R=0.5
#basket['custom_position'] = basket.apply(lambda x: custom_bond_position(x, basket,R), axis=1)

r1d = 4.33
basket.loc['1D']=pd.Series({
                            'id_isin': '1D',
                            'maturity': s_date+pd.offsets.Day(),
                            'int_acc_dt': None,
                            'issue_dt': s_date,
                            'coupon': 0,
                            'length': 1/act,
                            'ttm': 1/act,
                            'length _text': 'ID',
                            'y_mid': r1d,
                            'p_mid': 100*np.exp(-r1d*1/act/100),
                            'AI': 0,
                            'dirty': 100*np.exp(-r1d*1/act/100)
                        })

density=basket.groupby(lambda x: round(basket. loc[x, 'ttm'])) .apply (lambda x: len(x))
basket_density=basket.apply(lambda x: density.loc[round(x.ttm)], axis=1)

# P array
P0=basket['dirty']
P_array= P0.values
n_bonds= len(P)
# term to maturity
ttm=(basket['ttm']*act)
ttm_array=ttm.values

In [25]:
pyd = pd.DataFrame(columns = ['payment_date'])

def correct_coupon_date(x):
    t1=pd.offsets.BusinessDay(1)
    if x.day>20:
        # get × to month end
        x = x -t1 + pd.offsets.MonthEnd()
        return utils.next_business_day(x)

    else :
        # get × to day 15
        x = x.replace(day=15)
        return utils.next_business_day(x)
    

for cusip in tqdm(cusips) :
    payments=pd.date_range(basket.loc[cusip,'issue_dt'], basket.loc[cusip,'maturity']+pd.offsets.BDay(29), freq=pd.DateOffset (months=6), inclusive='right')
    payments=pd.DataFrame(index=[cusip for _ in range(len(payments))], data=payments,columns=['payment_date'])
    payments['payment_date']=payments['payment_date'].apply(correct_coupon_date)

    pyd=pd.concat( (pyd, payments)) 

  0%|          | 0/7 [00:00<?, ?it/s]C:\Users\hacen\AppData\Local\Temp\ipykernel_20180\2339485326.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pyd=pd.concat( (pyd, payments))
100%|██████████| 7/7 [00:00<00:00, 98.65it/s]

In [27]:
pyd = pyd['payment_date']

In [28]:
# payment dates
pyd.loc['1D'] = basket.loc['1D', 'maturity']
# see docs of 'construct_C'
C=construct_C(ttm, pyd, basket, s_date)

data=prep_data(P0, C, density, basket_density)

# max time to maturity in days
N=int(ttm.max())
# generate kerne] matrix
K=kr_model.generate_kernel_matrix(alpha, delta, N, N)

#** fit KR model
# KR ridge penalty term

dict_fit=kr_model.KR(    C=data['C'], # cashflow matrix
                        B=data['P'], # price vector
                        ridge=ridge, # ridge hyper-parameter
                        inv_w=data['inv_w'], #p.ones like(data['inv w']I. # inverse of the weichting vector
                        K=K, # kernel matrix.
                        start_curve=None
)

                    
r=dict_fit['y_solved']
g=dict_fit['g_solved']
c=data['C']
M=C.shape[0]
B_fitted=C@g[:C.shape[1]]
e=- (data['P']-B_fitted)*100
e=e/(data['dur']*data['P']/100)
np.abs(e).mean(), np.abs(e).std()
e=pd.Series(index=basket.index, data=e)

In [29]:
g

array([0.99988139, 0.99976284, 0.99964434, ..., 0.25107015, 0.25104242,
       0.25101471])

In [30]:
r

array([0.0432946 , 0.04328757, 0.04327926, ..., 0.04616017, 0.04615964,
       0.0461591 ])